In [54]:
import agentpy as ap  # Library for creating agents
import numpy as np  # NumPy library for numerical operations
import matplotlib.pyplot as plt  # Library for plotting
import seaborn as sns  # Library for statistical data visualization
from random import randint  # For generating random numbers
import IPython  # For displaying videos in the notebook
from matplotlib.animation import FuncAnimation  # For creating animations (used by agentpy)

In [55]:
class Road(ap.Agent):
    def setup(self):
        self.custom_id = 1 # Custom ID for the road agent
        self.direction_id = None # Direction ID for the road agent to be set within the model
        # 1= southbound, 2= northbound, 3= eastbound, 4= westbound, 5= intersection area
        self.is_spawn = np.nan # flag to check if the road agent is a spawn point, 10 if spawn point
        self.is_end = np.nan # flag to check if the road agent is an end point, 20 if end point


    def get_position(self):
        return self.model.grid.positions[self]

In [56]:
class Car(ap.Agent):
    def setup(self):
        self.custom_id = 2 # Custom ID for the car agent
        # Speed of the car, randomly starts between 2 and 3
        self.speed = randint(2, 3)
        #0 = stopped, 1 = 1 cell pero two time steps, 2 = 1 cell per time step, 3 = 2 cells per time step
        self.sight_distance = 3 # Sight distance of the car
        self.car_in_front = False # Flag to check if there is a car in front of the car
        self.time_alive = 0 # Time alive of the car


    def action(self):
        if self.time_alive == 5:
            self.model.grid.remove_agents(self)

        
    def get_position(self):
        return self.model.grid.positions[self]

In [57]:
class Traffic_Light(ap.Agent):
    def setup(self):
        self.custom_id = 3 # Custom ID for the traffic light agent
        self.state = 0 # State of the traffic light, 0 = red, 1 = green


        
    def get_position(self):
        return self.model.grid.positions[self]

In [58]:
class TraffcController(ap.Agent):
    def setuup(self):
        self.custom_id = 4 # Custom ID for the traffic controller agent
        self.traffic_lights = [] # List to store the traffic light agents
        self.timer = None # Interval for the traffic lights to change state

In [62]:
class IntersectionModel(ap.Model):
    def setup(self):
        self.spawn_points = [] # List to store the spawn points
        # Create grid
        self.grid = ap.Grid(self, (self.p.dimensions, self.p.dimensions))

        # Create roads
        self.setup_roads()

        # Create cars
        self.cars = [] # List to store the car agents


    def update_car_count(self): #Function to add cars to the grid
        # Adds cars to the grid until the max is reached or spawn points are full
        
        if np.random.rand() < self.p.car_spawn_chance:
        # Check if the number of cars has reached the maximum
            if len(self.cars) == self.p.max_cars:
                return # Return if the maximum number of cars has been reached
            
            # Check spawn points arent full
            uncovered_spawn_points = []
            for spawn_point in self.spawn_points:
                # Check if the spawn point is covered by a car
                if len(self.grid.agents[spawn_point.get_position()]) == 1:
                    uncovered_spawn_points.append(spawn_point) # Add to the list of uncovered spawn points
            
            if len(uncovered_spawn_points) == 0:
                return # Return if all spawn points are covered
        
            self.add_car()

    def add_car(self, spawn_points): #Function to add cars to the grid
        # Add a car to the grid
        car = ap.AgentList(self, 1, Car)
        
        # Randomly select a spawn point, if it is covered, try again
        spawn_point = self.spawn_points[randint(0, len(self.spawn_points) - 1)]




                    
    def step(self):
        pass

    def update(self):
        pass

    def end(self):
        pass

    def add_car(self):
        pass

    def setup_car(self, car):
        pass

    def setup_roads(self):
        # CREATE ROAD AGENTS
        # Create the intersection matrix
        self.intersection_matrix = self.create_intersection_matrix(self.p.dimensions, self.p.dimensions)
        #print(self.intersection_matrix)
    
        # Add road agents to the grid following the intersection matrix, adding each road agent to their respective direction
        # 1= southbound, 2= northbound, 3= eastbound, 4= westbound, 5= intersection area
        for i in range(self.p.dimensions):
            for j in range(self.p.dimensions):
                if self.intersection_matrix[i][j] != 0:
                    #print(i, j, int(self.intersection_matrix[i][j]))
                    road = ap.AgentList(self, 1, Road)
                    road[0].direction_id = int(self.intersection_matrix[i][j])
                    self.grid.add_agents(road, [(int(i), int(j))])


        # MARK SPAWN AND END POINTS
        self.mark_spawn_and_end_points()
                
    def create_intersection_matrix(self, n, m):
        # Initialize the matrix with zeros
        matrix = np.zeros((n, m))
        
        # Determine the central points
        center_n = n // 2 # center of rows/height
        center_m = m // 2 # center of columns/width
        
        # Adjust for even dimensions - to ensure the intersection is centered
        n_start = center_n - 1 if n % 2 == 0 else center_n
        m_start = center_m - 1 if m % 2 == 0 else center_m
        
        # Create intersection area
        for i in range(n_start, n_start+2):
            for j in range(m_start, m_start+2):
                matrix[i][j] = 5
        
        # Create lanes leading to the intersection
        # Southbound and Northbound lanes
        for i in range(n_start):
            matrix[i][m_start] = 1
            matrix[i][m_start+1] = 2
        for i in range(n_start+2, n):
            matrix[i][m_start] = 1
            matrix[i][m_start+1] = 2
        
        # Eastbound and Westbound lanes
        for j in range(m_start):
            matrix[n_start][j] = 4
            matrix[n_start+1][j] = 3
        for j in range(m_start+2, m):
            matrix[n_start][j] = 4
            matrix[n_start+1][j] = 3
        
        # 1= southbound, 2= northbound, 3= eastbound, 4= westbound, 5= intersection area
        return matrix
        
    def mark_spawn_and_end_points(self):
        dimensions = self.p.dimensions
        for i in range(dimensions):
            for j in range(dimensions):
                direction_id = int(self.intersection_matrix[i][j])
                if direction_id != 0:
                    road_agents = self.grid.agents[(i, j)]
                    if not road_agents:
                        continue  # No agent at this position, should not happen but check for safety
                    
                    road_agent = road_agents  # Assuming one agent per grid cell
                    # Check if the agent's position is at the edge and matches the direction for a spawn point
                    if (direction_id == 1 and i == 0) or (direction_id == 2 and i == dimensions - 1) or \
                    (direction_id == 3 and j == 0) or (direction_id == 4 and j == dimensions - 1):
                        road_agent.is_spawn = 10
                        self.spawn_points.append(road_agent)
                        
                    # Check if the agent's position is at the opposite edge and matches the direction for an end point
                    if (direction_id == 1 and i == dimensions - 1) or (direction_id == 2 and i == 0) or \
                    (direction_id == 3 and j == dimensions - 1) or (direction_id == 4 and j == 0):
                        road_agent.is_end = 20
                        



In [63]:
parameters={
    'dimensions': 30,  # Dimensions of the grid
    'steps': 100,  # Number of steps to run the model
    'max_cars': 100, # Maximum number of cars
    'spawn_rate': 0.2 # Rate of car spawn, chance of car spawn per step 
}

model = IntersectionModel(parameters)
results = model.run()

#print(model.intersection_agent)


Completed: 100 steps
Run time: 0:00:00.143438
Simulation finished


In [64]:
def animation_plot(model, ax):
    # get all road agents
    attr_grid = model.grid.attr_grid('custom_id')

    # get all spawn and end points
    spawn_grid = model.grid.attr_grid('is_spawn')
    end_grid = model.grid.attr_grid('is_end')

    #Create masks for spawn and end points
    spawn_mask = ~np.isnan(spawn_grid)
    end_mask = ~np.isnan(end_grid)

    #overlay spawn and end points over road agent grid
    attr_grid[spawn_mask] = spawn_grid[spawn_mask]
    attr_grid[end_mask] = end_grid[end_mask]
    
    color_dict = {1: 'gray', 2: 'red', 10: 'green', 20: 'yellow', 5: 'black', None: 'white'}

    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

    ax.set_title(f"Road Generation Model\n"
                 f"intersection = black, southbound = blue,\n"
                 f"northbound = red, eastbound = green, westbound = yellow")

fig, ax = plt.subplots()
animation = ap.animate(model, fig, ax, animation_plot)

IPython.display.HTML(animation.to_jshtml(fps=2))